In [1]:
import numpy as np
import pandas as pd
import pickle

# 保存训练集

In [2]:
df = pd.read_csv("../data/CVT/train.csv")

In [3]:
len(df)

340000

In [7]:
df = df.loc[:, ['a', 'b', 'c']]

In [ ]:
df

In [9]:
numpy_train = df.to_numpy()

In [ ]:
numpy_train

In [11]:
import pickle

with open("../data/CVT/cvt_train.pkl", "wb") as f:
    pickle.dump(numpy_train, f)

# 测试集数据处理

In [12]:
import numpy as np
import pandas as pd
import pickle

df = pd.read_csv("../data/CVT/test.csv")

# 假设我们有一个标签列表，初始时全是 0，表示正常数据
labels = np.zeros(len(df))


# 设置异常数据的长度和数量
num_anomalies = 150  # 需要插入异常数据的数量
anomaly_length = 50  # 每个异常区段的长度（例如50行）

# 生成异常数据的起始位置
np.random.seed(2024)  # 设置随机种子以确保结果可重现
anomaly_start_indices = np.random.choice(len(df) - anomaly_length, num_anomalies, replace=False)

# 对每个异常区段进行处理
for start_idx in anomaly_start_indices:
    # 在这段连续区间内加入异常，假设我们在第0列数据上增加0.3%模拟异常
    df.iloc[start_idx:start_idx + anomaly_length, 0] *= 1.002  # 这里模拟在第0列增加1%的异常
    labels[start_idx:start_idx + anomaly_length] = 1  # 将这段区间标记为异常

# 将标签添加为 DataFrame 的一列
df['Label'] = labels

df['Label'] = df['Label'].astype(int)
print(df['Label'].sum())
df_label = df.iloc[:,-1]
df_test = df.iloc[:,:-1]

7063


In [13]:
numpy_test = df_test.to_numpy()
numpy_label = df_label.to_numpy()

with open("../data/CVT/cvt_test_03.pkl", "wb") as f:
    pickle.dump(numpy_test, f)

with open("../data/CVT/cvt_test_label_03.pkl", "wb") as f:
    pickle.dump(numpy_label, f)

# 第二种方法

In [19]:
import numpy as np
import pandas as pd
import pickle

# 假设有一个原始数据框
np.random.seed(2024)  # 设置随机种子以确保可复现
df = pd.read_csv("../data/CVT/test.csv")

n = df.shape[0]

# 随机选择多个误差时间段
num_error_segments = np.random.randint(200, 220)  # 随机选择误差段数，200到250段

# 用于标记误差的列
df['label'] = 0

# 存储已使用的误差段区间
occupied_intervals = []

# 设置每列发生误差的概率
error_probabilities = {'a': 0.3, 'b': 0.5, 'c': 0.6}  # 'a' 列 60% 概率，'b' 列 50% 概率，'c' 列 30% 概率

# 生成误差段
error_segments = []
for _ in range(num_error_segments):
    # 随机选择误差段的开始位置和持续时间
    error_start = np.random.randint(0, n - 100)  # 错误开始位置
    error_length = np.random.randint(80, 100)  # 错误持续时间

    # 记录这个误差段
    occupied_intervals.append((error_start, error_length))

    # 根据设置的概率选择需要施加误差的列
    columns_to_error = []
    for col, prob in error_probabilities.items():
        if np.random.rand() < prob:  # 根据概率决定是否施加误差
            columns_to_error.append(col)

    # 如果没有列被选中，则强制选中'a'列
    if len(columns_to_error) == 0:
        columns_to_error.append('a')

    # 随机选择误差幅度，在 0.2% 到 0.3% 之间浮动
    error_percentage = np.random.uniform(0.002, 0.003)

    # 记录误差段
    error_segments.append((error_start, error_length, columns_to_error, error_percentage))

# 对每个误差段施加误差
for error_start, error_length, columns_to_error, error_percentage in error_segments:
    for col in columns_to_error:
        df.loc[error_start:error_start + error_length, [col]] *= (1 + error_percentage)
    df.loc[error_start:error_start + error_length, 'label'] = 1


In [20]:
df_label = df.iloc[:,-1]
df_test = df.iloc[:,:-1]

numpy_test = df_test.to_numpy()
numpy_label = df_label.to_numpy()

with open("../data/CVT/cvt_test_00.pkl", "wb") as f:
    pickle.dump(numpy_test, f)

with open("../data/CVT/cvt_test_label_00.pkl", "wb") as f:
    pickle.dump(numpy_label, f)